In [3]:
import numpy as np

import torch

import torch.nn as nn

import torch.nn.functional as F

import matplotlib.pyplot as plt

import cv2

from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split

from torch.nn import CrossEntropyLoss

from torch.optim import Adam

from torch.utils.data import DataLoader, TensorDataset, random_split

from torchvision.datasets.mnist import MNIST

from torchvision.transforms import ToTensor

from tqdm import tqdm

from torchvision import transforms

from torchvision.datasets import Imagenette

from torch.utils.data.sampler import SubsetRandomSampler

from mpl_toolkits.axes_grid1 import ImageGrid

from torchvision.transforms import v2

import torchvision

import math

import os

import pickle



BATCH_SIZE = 128

CLASS_NUM = 10

In [2]:
!rm -rf /kaggle/working/encoder_student.pt
!rm -rf /kaggle/working/Pickles/*
!rm -rf /kaggle/working/classifier.pt

# Data

In [4]:
class CustomTensorDataset(Dataset):

    def __init__(self, X, y, transform):

        self.X = X

        self.y = y

        self.transform = transform







    def __len__(self):

        return len(self.X)







    def __getitem__(self, idx):

        image, label = self.X[idx], self.y[idx]

        image = self.transform(image)

        return image, label

In [5]:
train_transform = transforms.Compose([

    v2.RandAugment(5,14),

    v2.RandomCrop(224, padding=16),

    v2.RandomHorizontalFlip(p=0.5),

    v2.RandomRotation(degrees=10),

    v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    
    v2.ToDtype(torch.float32, scale=True),

    v2.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),

])



test_transform = transforms.Compose([

    v2.ToDtype(torch.float32, scale=True),

    v2.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),

])







train_dataset  = Imagenette('./data/',split='train',transform=transforms.Compose([v2.Resize((224, 224)), v2.ToImage()]),download=False)



test_dataset   = Imagenette('./data/',split='val',transform=transforms.Compose([v2.Resize((224, 224)), v2.ToImage()]),download=False)







print('loading data on ram')



train_data   = torch.stack([train_dataset[i][0] for i in range(len(train_dataset))])

train_labels = torch.tensor([train_dataset[i][1] for i in range(len(train_dataset))])

test_data    = torch.stack([test_dataset[i][0] for i in range(len(test_dataset))])

test_labels  = torch.tensor([test_dataset[i][1] for i in range(len(test_dataset))])

indices = torch.randperm(len(train_data))







indices_train = indices[:int(0.9 * len(train_data))]

indices_valid = indices[int(0.9 * len(train_data)):]







valid_data   = train_data[indices_valid]

train_data   = train_data[indices_train]







valid_labels = train_labels[indices_valid]

train_labels = train_labels[indices_train]







train_dataset = CustomTensorDataset(train_data, train_labels, transform=train_transform)

valid_dataset = CustomTensorDataset(valid_data, valid_labels, transform=test_transform)

test_dataset  = CustomTensorDataset(test_data, test_labels, transform=test_transform)







batch_size = 128



train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, pin_memory=True, num_workers=4, shuffle=True)

valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, pin_memory=True, num_workers=4, shuffle=True)

test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

loading data on ram


# Network

In [5]:
class SupConLoss(nn.Module):

    """Supervised Contrastive Learning: https://arxiv.org/pdf/2004.11362.pdf.

    It also supports the unsupervised contrastive loss in SimCLR"""

    def __init__(self, temperature=0.07, contrast_mode='all',

                 base_temperature=0.07):



        super(SupConLoss, self).__init__()

        self.temperature = temperature

        self.contrast_mode = contrast_mode

        self.base_temperature = base_temperature







    def forward(self, features, labels=None, mask=None):

        """Compute loss for model. If both `labels` and `mask` are None,

        it degenerates to SimCLR unsupervised loss:

        https://arxiv.org/pdf/2002.05709.pdf



        Args:

            features: hidden vector of shape [bsz, n_views, ...].

            labels: ground truth of shape [bsz].

            mask: contrastive mask of shape [bsz, bsz], mask_{i,j}=1 if sample j

                has the same class as sample i. Can be asymmetric.



        Returns:

            A loss scalar.

        """



        device = (torch.device('cuda')

                  if features.is_cuda

                  else torch.device('cpu'))



        if len(features.shape) < 3:

            raise ValueError('`features` needs to be [bsz, n_views, ...],'

                             'at least 3 dimensions are required')



        if len(features.shape) > 3:

            features = features.view(features.shape[0], features.shape[1], -1)







        batch_size = features.shape[0]

        if labels is not None and mask is not None:

            raise ValueError('Cannot define both `labels` and `mask`')

        elif labels is None and mask is None:

            mask = torch.eye(batch_size, dtype=torch.float32).to(device)

        elif labels is not None:

            labels = labels.contiguous().view(-1, 1)

            if labels.shape[0] != batch_size:

                raise ValueError('Num of labels does not match num of features')

            mask = torch.eq(labels, labels.T).float().to(device)

        else:

            mask = mask.float().to(device)







        contrast_count = features.shape[1]

        contrast_feature = torch.cat(torch.unbind(features, dim=1), dim=0)

        if self.contrast_mode == 'one':

            anchor_feature = features[:, 0]

            anchor_count = 1

        elif self.contrast_mode == 'all':

            anchor_feature = contrast_feature

            anchor_count = contrast_count

        else:

            raise ValueError('Unknown mode: {}'.format(self.contrast_mode))







        # compute logits

        anchor_dot_contrast = torch.div(

            torch.matmul(anchor_feature, contrast_feature.T),

            self.temperature)



        # for numerical stability



        logits_max, _ = torch.max(anchor_dot_contrast, dim=1, keepdim=True)

        logits = anchor_dot_contrast - logits_max.detach()





        # tile mask

        mask = mask.repeat(anchor_count, contrast_count)

        # mask-out self-contrast cases



        logits_mask = torch.scatter(

            torch.ones_like(mask),

            1,

            torch.arange(batch_size * anchor_count).view(-1, 1).to(device),

            0

        )

        mask = mask * logits_mask

        # compute log_prob

        exp_logits = torch.exp(logits) * logits_mask

        log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))

        # compute mean of log-likelihood over positive

        # modified to handle edge cases when there is no positive pair

        # for an anchor point.

        # Edge case e.g.:-

        # features of shape: [4,1,...]

        # labels:            [0,1,1,2]

        # loss before mean:  [nan, ..., ..., nan]

        mask_pos_pairs = mask.sum(1)

        mask_pos_pairs = torch.where(mask_pos_pairs < 1e-6, 1, mask_pos_pairs)

        mean_log_prob_pos = (mask * log_prob).sum(1) / mask_pos_pairs



        # loss

        loss = - (self.temperature / self.base_temperature) * mean_log_prob_pos

        loss = loss.view(anchor_count, batch_size).mean()



        return loss



def normalize1(logit):
    mean = logit.mean(dim=-1, keepdims=True)
    stdv = logit.std(dim=-1, keepdims=True)
    return (logit - mean) / (1e-7 + stdv)
 



class LogitStandardizationLoss(nn.Module):

    def __init__(self):
        super(LogitStandardizationLoss, self).__init__()


    def forward(self, logits_teacher_in, logits_student_in, temperature = 2):
        logits_student = normalize1(logits_student_in) 
        logits_teacher = normalize1(logits_teacher_in)
        log_pred_student = F.log_softmax(logits_student / temperature, dim=1)
        pred_teacher = F.softmax(logits_teacher / temperature, dim=1)
        loss_kd = F.kl_div(log_pred_student, pred_teacher, reduction="none").sum(1).mean()
        loss_kd *= temperature**2
        return loss_kd

    

In [36]:
from torchvision.models import resnet101, resnet18



encoder_teacher = resnet101()

encoder_student = resnet18()

encoder_teacher.fc = nn.Identity()

encoder_student.fc = nn.Identity()





class NormalizeLayer(nn.Module):

    def __init__(self):

        super(NormalizeLayer, self).__init__()



    def forward(self, x):

        # Normalize the input tensor `x` using the specified parameters

        return F.normalize(x)



    



    

projection_head_teacher = nn.Sequential(

      nn.Flatten(),

      nn.Linear(2048,2048),

      nn.ReLU(),

      nn.Linear(2048,128),

      NormalizeLayer()

)





projection_head_student = nn.Sequential(

      nn.Flatten(),

      nn.Linear(512,512),

      nn.ReLU(),

      nn.Linear(512,128),

      NormalizeLayer()

)



class Classifier(nn.Module):
    def __init__(self, input_features, num_classes):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_features, 128)  # Reduced size
        self.batch_norm = nn.BatchNorm1d(128)  # Batch normalization for stability
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.4)  # Moderate dropout to prevent overfitting
        self.fc2 = nn.Linear(128, num_classes)  # Final classification layer
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.batch_norm(x)  # Normalize after the first linear layer
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)  # No activation for the output (handled by loss function)
        return x


# Classifier = nn.Linear(512, CLASS_NUM)

# Classifier.to('cuda')



encoder_teacher.to('cuda')

encoder_student.to('cuda')

projection_head_teacher.to('cuda')

projection_head_student.to('cuda')











encoder_teacher.load_state_dict(torch.load('/kaggle/working/encoder_teacher.pt'))

projection_head_teacher.load_state_dict(torch.load('/kaggle/working/projection_head_teacher.pt'))





encoder_teacher.eval()

projection_head_teacher.eval()





for name, param in encoder_teacher.named_parameters():

    param.requires_grad = False



    



for name, param in projection_head_teacher.named_parameters():

    param.requires_grad = False

/tmp/ipykernel_30/330953206.py:115: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder_teacher.load_state_dict(torch.load('/kaggle/working/encoder_teacher.pt'))
/tmp/ipyk

# Training

In [7]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)



criterion1 = LogitStandardizationLoss()

criterion2 = SupConLoss()







learning_rate = 0.0006

lr_decay_rate = 0.1

n_epochs = 300







# specify optimizer



optimizer = optim.AdamW(list(encoder_student.parameters()) + list(projection_head_student.parameters()), lr=learning_rate)



def adjust_learning_rate(optimizer, epoch):

    lr = np.copy(learning_rate)

    eta_min = lr * (lr_decay_rate ** 3)

    lr = eta_min + (lr - eta_min) * (

            1 + math.cos(math.pi * epoch / n_epochs)) / 2

    for param_group in optimizer.param_groups:

        param_group['lr'] = lr

In [8]:
valid_loss_min = np.Inf 
train_losses_dict_1 = {}
valid_losses_dict_1 = {}
train_losses_dict_2 = {}
valid_losses_dict_2 = {}




for epoch in range(1, n_epochs+1):

    adjust_learning_rate(optimizer, epoch)

    # keep track of training and validation loss

    train_loss1 = 0.0

    train_loss2 = 0.0

    valid_loss1 = 0.0

    valid_loss2 = 0.0



    ###################



    # train the model #



    ###################



    encoder_student.train()

    projection_head_student.train()

    for batch in tqdm(train_loader):

        x, y = batch[0].cuda(), torch.flatten(batch[1].cuda(),0)

        optimizer.zero_grad()

        with torch.no_grad():

            output_teacher = projection_head_teacher(encoder_teacher(x))

        output_student = projection_head_student(encoder_student(x))

        output_student_cat = torch.cat([F.normalize(output_student).unsqueeze(1), F.normalize(output_student).unsqueeze(1)], dim=1)

        loss1 = criterion1(output_teacher.detach(), output_student)

        loss2 = criterion2(output_student_cat, y)

        loss   = loss1 + 0.15*loss2

        loss.backward()

        optimizer.step()

        train_loss1 += loss1.item()*x.size(0)

        train_loss2 += loss2.item()*x.size(0)



            



    ######################    



    # validate the model #



    ######################



    



    encoder_student.eval()

    projection_head_student.eval()

    with torch.no_grad():

        for batch in tqdm(valid_loader):

            x, y = batch[0].cuda(), torch.flatten(batch[1].cuda(),0)

            output_teacher = projection_head_teacher(encoder_teacher(x))

            output_student = projection_head_student(encoder_student(x))

            output_student_cat = torch.cat([F.normalize(output_student).unsqueeze(1), F.normalize(output_student).unsqueeze(1)], dim=1)

            loss1 = criterion1(output_teacher.detach(), output_student)

            loss2 = criterion2(output_student_cat, y)

            valid_loss1 += loss1.item()*x.size(0)

            valid_loss2 += loss2.item()*x.size(0)







    # calculate average losses



    train_loss1 = train_loss1/len(train_labels)

    train_loss2 = train_loss2/len(train_labels)

    valid_loss1 = valid_loss1/len(valid_labels)

    valid_loss2 = valid_loss2/len(valid_labels)

    total_valid = 1*valid_loss1 + 0.15* valid_loss2

    train_losses_dict_1[epoch] = train_loss1
    train_losses_dict_2[epoch] = train_loss2
    valid_losses_dict_1[epoch] = valid_loss1
    valid_losses_dict_2[epoch] = valid_loss2



    # print training/validation statistics 

    print('Epoch: {} \tTraining Loss: {:.5f},{:.5f} \tValidation Loss: {:.5f},{:.5f}'.format(

        epoch, train_loss1, train_loss2, valid_loss1, valid_loss2))



        



    # save model if validation loss has decreased



    if total_valid <= valid_loss_min:

        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(

        valid_loss_min,

        total_valid))

        torch.save(encoder_student.state_dict(), 'encoder_student.pt')

        valid_loss_min = np.copy(total_valid)



        
os.makedirs('/kaggle/working/Pickles', exist_ok=True)


train_losses_1_path = '/kaggle/working/Pickles/train_losses_encoder_resnet101_18.pkl'
valid_losses_1_path = '/kaggle/working/Pickles/valid_losses_encoder_resnet101_18.pkl'
train_losses_2_path = '/kaggle/working/Pickles/train_losses_encoder_resnet18.pkl'
valid_losses_2_path = '/kaggle/working/Pickles/valid_losses_encoder_resnet18.pkl'


if not os.path.exists(train_losses_1_path):
    fd = os.open('/kaggle/working/Pickles/train_losses_encoder_resnet101_18.pkl', os.O_CREAT)
    os.close(fd)
    with open(train_losses_1_path, 'wb') as f:
        pickle.dump(train_losses_dict_1, f)
    print("Training losses saved!")
else:
    with open(train_losses_1_path, 'wb') as f:
        pickle.dump(train_losses_dict_1, f)
    print("Training losses saved!")


if not os.path.exists(valid_losses_1_path):
    fd = os.open('/kaggle/working/Pickles/valid_losses_encoder_resnet101_18.pkl', os.O_CREAT)
    os.close(fd)
    with open(valid_losses_1_path, 'wb') as f:
        pickle.dump(valid_losses_dict_1, f)
    print("Validation losses saved!")
else:
    with open(valid_losses_1_path, 'wb') as f:
        pickle.dump(valid_losses_dict_1, f)
    print("Validation losses saved!")


if not os.path.exists(train_losses_2_path):
    fd = os.open('/kaggle/working/Pickles/train_losses_encoder_resnet18.pkl', os.O_CREAT)
    os.close(fd)
    with open(train_losses_2_path, 'wb') as f:
        pickle.dump(train_losses_dict_2, f)
    print("Training losses saved!")
else:
    with open(train_losses_2_path, 'wb') as f:
        pickle.dump(train_losses_dict_2, f)
    print("Training losses saved!")


if not os.path.exists(valid_losses_2_path):
    fd = os.open('/kaggle/working/Pickles/valid_losses_encoder_resnet18.pkl', os.O_CREAT)
    os.close(fd)
    with open(valid_losses_2_path, 'wb') as f:
        pickle.dump(valid_losses_dict_2, f)
    print("Validation losses saved!")
else:
    with open(valid_losses_2_path, 'wb') as f:
        pickle.dump(valid_losses_dict_2, f)
    print("Validation losses saved!")

100%|██████████| 8/8 [00:03<00:00,  2.42it/s]


Epoch: 1 	Training Loss: 0.30316,5.53176 	Validation Loss: 0.28689,5.47727
Validation loss decreased (inf --> 1.108483).  Saving model ...


100%|██████████| 8/8 [00:03<00:00,  2.39it/s]


Epoch: 2 	Training Loss: 0.26276,5.48003 	Validation Loss: 0.26407,5.33430
Validation loss decreased (1.108483 --> 1.064218).  Saving model ...


100%|██████████| 8/8 [00:03<00:00,  2.41it/s]


...

Epoch: 298 	Training Loss: 0.01429,3.35990 	Validation Loss: 0.02933,3.72299


100%|██████████| 8/8 [00:03<00:00,  2.35it/s]


Epoch: 299 	Training Loss: 0.01366,3.36180 	Validation Loss: 0.02889,3.71168


100%|██████████| 8/8 [00:03<00:00,  2.37it/s]

Epoch: 300 	Training Loss: 0.01278,3.34175 	Validation Loss: 0.02905,3.72776
Training losses saved!
Validation losses saved!
Training losses saved!
Validation losses saved!


In [58]:
encoder_student.load_state_dict(torch.load('encoder_student.pt'))

classifier = Classifier(input_features=512, num_classes=CLASS_NUM)

classifier.to('cuda')





for name, param in encoder_student.named_parameters():

    param.requires_grad = False



    

encoder_student.eval()      

/tmp/ipykernel_30/4116212640.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder_student.load_state_dict(torch.load('encoder_student.pt'))


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [50]:
! pip install lion-pytorch

In [59]:
import torch.optim as optim
from lion_pytorch import Lion

# specify loss function (categorical cross-entropy)

criterion = nn.CrossEntropyLoss()



# specify optimizer

optimizer = Lion(classifier.parameters(), lr=0.0001, weight_decay=1e-4)

In [60]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Add a learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)


In [61]:
# Early stopping parameters
early_stop_patience = 5
no_improvement_epochs = 0

# Track best validation loss
valid_loss_min = np.Inf
train_losses_dict = {}
valid_losses_dict = {}

# Training loop
n_epochs = 50
for epoch in range(1, n_epochs + 1):
    train_loss = 0.0
    valid_loss = 0.0

    # Training
    classifier.train()
    for batch in tqdm(train_loader):
        x, y = batch[0].cuda(), batch[1].cuda()
        optimizer.zero_grad()
        with torch.no_grad():  # Prevent gradients for encoder
            encoded_features = encoder_student(x)
        output = classifier(encoded_features.detach())
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)

    # Validation
    classifier.eval()
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            x, y = batch[0].cuda(), batch[1].cuda()
            encoded_features = encoder_student(x)
            output = classifier(encoded_features)
            loss = criterion(output, y)
            valid_loss += loss.item() * x.size(0)

    # Average losses
    train_loss /= len(train_labels)
    valid_loss /= len(valid_labels)
    train_losses_dict[epoch] = train_loss
    valid_losses_dict[epoch] = valid_loss

    print(f'Epoch {epoch}: Train Loss: {train_loss:.6f}, Valid Loss: {valid_loss:.6f}')

    # Save model if validation loss improves
    if valid_loss < valid_loss_min:
        print(f'Validation loss decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(classifier.state_dict(), f'classifier_epoch_{epoch}_val_loss_{valid_loss:.4f}.pt')
        valid_loss_min = valid_loss
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1

    # Check early stopping
    if no_improvement_epochs >= early_stop_patience:
        print("Early stopping triggered.")
        break

    # Adjust learning rate based on validation loss
    scheduler.step(valid_loss)

# Save loss dictionaries as pickle files
os.makedirs('/kaggle/working/Pickles', exist_ok=True)
train_losses_path = '/kaggle/working/Pickles/train_losses_projection_head_student.pkl'
valid_losses_path = '/kaggle/working/Pickles/valid_losses_projection_head_student.pkl'

with open(train_losses_path, 'wb') as f:
    pickle.dump(train_losses_dict, f)
print("Training losses saved!")

with open(valid_losses_path, 'wb') as f:
    pickle.dump(valid_losses_dict, f)
print("Validation losses saved!")

100%|██████████| 8/8 [00:01<00:00,  6.75it/s]


Epoch 1: Train Loss: 1.288653, Valid Loss: 0.270264
Validation loss decreased (inf --> 0.270264). Saving model...


100%|██████████| 8/8 [00:01<00:00,  6.73it/s]


Epoch 2: Train Loss: 0.680599, Valid Loss: 0.188096
Validation loss decreased (0.270264 --> 0.188096). Saving model...


100%|██████████| 8/8 [00:01<00:00,  6.55it/s]


...

Epoch 13: Train Loss: 0.128345, Valid Loss: 0.056750


100%|██████████| 8/8 [00:01<00:00,  6.79it/s]


Epoch 14: Train Loss: 0.143221, Valid Loss: 0.060282


100%|██████████| 8/8 [00:01<00:00,  6.43it/s]

Epoch 15: Train Loss: 0.144345, Valid Loss: 0.060507
Early stopping triggered.
Training losses saved!
Validation losses saved!


In [ ]:
classifier.load_state_dict(torch.load('classifier.pt'))

# Eval

In [62]:
encoder_student.eval()

classifier.eval()



losses       = 0

num_corrects = 0

n            = 0



with torch.no_grad():

    for batch in test_loader:

        x = batch[0].to('cuda')

        y = batch[1].to('cuda')







        n += len(y)

        output = classifier(encoder_student(x))

        num_corrects += torch.sum(torch.argmax(output, dim=1) == y).cpu().item()







num_corrects/n

0.9245859872611465

In [63]:
import pickle
import matplotlib.pyplot as plt

# Helper function to load data
def load_pickle(filepath):
    try:
        with open(filepath, 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"File not found: {filepath}")
    except Exception as e:
        raise RuntimeError(f"Error loading file {filepath}: {e}")

# Load data for all models
train_losses_resnet101 = load_pickle('/kaggle/working/Pickles/train_losses_encoder_resnet101_18.pkl')
valid_losses_resnet101 = load_pickle('/kaggle/working/Pickles/valid_losses_encoder_resnet101_18.pkl')

train_losses_resnet18 = load_pickle('/kaggle/working/Pickles/train_losses_encoder_resnet18.pkl')
valid_losses_resnet18 = load_pickle('/kaggle/working/Pickles/valid_losses_encoder_resnet18.pkl')

train_losses_projection_head = load_pickle('/kaggle/working/Pickles/train_losses_projection_head_student.pkl')
valid_losses_projection_head = load_pickle('/kaggle/working/Pickles/valid_losses_projection_head_student.pkl')

# Prepare data for plotting
def prepare_data(train_losses, valid_losses):
    if isinstance(train_losses, list) and isinstance(valid_losses, list):
        epochs = list(range(1, len(train_losses) + 1))
    elif isinstance(train_losses, dict) and isinstance(valid_losses, dict):
        common_epochs = sorted(set(train_losses.keys()) & set(valid_losses.keys()))
        if not common_epochs:
            raise ValueError("No common epochs found in train_losses and valid_losses.")
        train_losses = [train_losses[epoch] for epoch in common_epochs]
        valid_losses = [valid_losses[epoch] for epoch in common_epochs]
        epochs = common_epochs
    else:
        raise TypeError("train_losses and valid_losses must both be lists or both be dictionaries.")
    
    # Convert all loss values to float
    try:
        train_losses = [float(loss) for loss in train_losses]
        valid_losses = [float(loss) for loss in valid_losses]
    except ValueError as e:
        raise ValueError("Non-numeric loss values found.") from e
    
    return epochs, train_losses, valid_losses

# Prepare data for each model
epochs_resnet101, train_resnet101, valid_resnet101 = prepare_data(train_losses_resnet101, valid_losses_resnet101)
epochs_resnet18, train_resnet18, valid_resnet18 = prepare_data(train_losses_resnet18, valid_losses_resnet18)
epochs_proj_head, train_proj_head, valid_proj_head = prepare_data(train_losses_projection_head, valid_losses_projection_head)

# Plot the data
plt.figure(figsize=(18, 8))

# Plot for ResNet-101
plt.subplot(1, 3, 1)
plt.plot(epochs_resnet101, train_resnet101, label='Training Loss', marker='o', linestyle='-')
plt.plot(epochs_resnet101, valid_resnet101, label='Validation Loss', marker='s', linestyle='--')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('ResNet-101-18 Loss')
plt.legend()
plt.grid(True)

# Plot for ResNet-18
plt.subplot(1, 3, 2)
plt.plot(epochs_resnet18, train_resnet18, label='Training Loss', marker='o', linestyle='-')
plt.plot(epochs_resnet18, valid_resnet18, label='Validation Loss', marker='s', linestyle='--')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('ResNet-18 Loss')
plt.legend()
plt.grid(True)

# Plot for Projection Head
plt.subplot(1, 3, 3)
plt.plot(epochs_proj_head, train_proj_head, label='Training Loss', marker='o', linestyle='-')
plt.plot(epochs_proj_head, valid_proj_head, label='Validation Loss', marker='s', linestyle='--')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Projection Head Loss')
plt.legend()
plt.grid(True)

# Display the plots
plt.tight_layout()
plt.show()
